In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import linalg as lg
import time
from tqdm import tqdm

from lightonopu.opu import OPU
from lightonml.random_projections.opu import OPURandomMapping

In [2]:
def cconj(A):
    return np.conj(A.T)

def do_MDS(D, number_of_anchors):
    m = number_of_anchors
    J = np.eye(m + 2) - 1. / (m + 2) * np.ones((m + 2, m + 2))
    G = -1/2 * np.dot(J, D).dot(J)
    U, s, VT = np.linalg.svd(G)
    Z_est_R2 = np.dot(np.diag(np.sqrt(s[:2])), VT[:2, :])
    Z_est_cpx = Z_est_R2[0, :] + 1j*Z_est_R2[1, :]
    
    # translate the origin back at (0, 0)
    Z_est_cpx -= Z_est_cpx[m + 1]
    
    return Z_est_cpx

def ortho_procrustes(fixed, modify):
    fixed = np.vstack ((np.real(fixed[1:]), np.imag(fixed[1:])))
    modify = np.vstack ((np.real(modify), np.imag(modify)))
    original = modify.copy()
    modify = modify[:,1:]
    fixed_mean = (np.mean(fixed, axis=1)).reshape([-1,1])
    fixed -= fixed_mean
    modify_mean = (np.mean(modify, axis=1)).reshape([-1,1])
    modify -= modify_mean
    M = fixed @ modify.T
    u, s, vh = np.linalg.svd(M)
    R = u @ vh
    original = R @ (original - modify_mean @ np.ones(
        [1, original.shape[1]])) + fixed_mean@np.ones([1, original.shape[1]])
    return original[0] + 1j*original[1]

def make_D_ensembles(y, number_of_anchors):
    num_elements = int((number_of_anchors+2)* (number_of_anchors+1) * 0.5)
    
    trials = y.shape[1]
    dim = number_of_anchors+2
    all_D_oracles_x = np.zeros([trials, dim, dim])
    
    ind = np.triu_indices(all_D_oracles_x[0].shape[0], k=1)
    for i in range(trials):
        data = y[0:num_elements,i]
        all_D_oracles_x[i][ind] = data
        all_D_oracles_x[i] += all_D_oracles_x[i].T
        
    return all_D_oracles_x

def interfere_with_anchors(n, x, anchors):
    interfered = anchors - x
    interfered = np.vstack((interfered, x)) # x with zero (zero is less than x so subtract the other way)
    
    anchors = np.vstack((anchors, np.zeros(n))) # zero is an anchor too
    
    for i in range(anchors.shape[0]-1):
        diffs = anchors[i] - anchors[1+i:]
        interfered = np.vstack((interfered, diffs))
    
    return interfered

In [3]:
def get_OPU_measurements(opu_input, num_rand_proj):
    opu = OPU(500, 100) # exposure needs to be chosen so that there is no saturation.
    mapping = OPURandomMapping(opu, n_components=num_rand_proj)
    y = mapping.fit_transform(opu_input.astype('uint8'))
    print (y.shape)
    print (np.max(y))
    print (np.min(y))
    
    return y

def make_anchors(X, number_of_anchors):
    X_sum = np.sum(X.copy(), axis=0)
    X_sum[X_sum>0] = 1
    
    anchors = np.zeros([number_of_anchors, X.shape[1]])
    
    anchor_p = [0.85,0.15]
    
    anchors[0] = np.random.choice([0,1], size=X.shape[1], p=anchor_p) + X_sum
    
    for i in range(1, number_of_anchors):
        anchors[i] = np.random.choice([0,1], size=X.shape[1], p=anchor_p) + anchors[i-1]

    anchors[anchors>0] = 1
    anchors = anchors[::-1] # for my convenience
    
    return anchors

def opu_projection(X, k):   
    number_of_anchors = 5
    
    anchors = make_anchors(X, number_of_anchors)

    x = np.zeros([1, X.shape[1]])
    opu_input = interfere_with_anchors(X.shape[1], x, anchors)
    anchors_input_size = opu_input.shape[0]
    
    for i in range(X.shape[0]):
        x = X[i]#.reshape([1,-1])
        opu_input = np.vstack((opu_input, interfere_with_anchors(X.shape[1], x, anchors)))
    
    num_of_rows_in_A = k
    print('Getting OPU data')
    y_quant = get_OPU_measurements(opu_input, num_of_rows_in_A)
    print('Got OPU data')
    
    # localise anchors
    anchor_positions = np.zeros([num_of_rows_in_A, number_of_anchors+2]).astype('complex128')
    all_D_quant = make_D_ensembles(y_quant[:anchors_input_size], number_of_anchors)
    for i in range(num_of_rows_in_A):
        anchor_positions[i] = do_MDS(all_D_quant[i], number_of_anchors)
        
    # localise other points
    results = np.ones([X.shape[0], num_of_rows_in_A]).astype('complex128')
    for i in range(X.shape[0]):
        all_D_quant = make_D_ensembles(y_quant[(i+1)*anchors_input_size:(i+2)*anchors_input_size], number_of_anchors)
        for row in range(num_of_rows_in_A):
            recovered_points = do_MDS(all_D_quant[row], number_of_anchors)
            recovered_points = ortho_procrustes(anchor_positions[row], recovered_points)
            results[i, row] = recovered_points[0]
    
    return results

def randsvd(M, k):
    m, n = M.shape
    
    Y = opu_projection(M, k)
    Y = cconj(Y)
    Y = np.vstack((np.real(Y), np.imag(Y)))
    Y = cconj(Y)
    
    Q = lg.orth(Y)
    B = (cconj(Q)) @ M
    U_tilde, s, Vh = lg.svd(B, full_matrices=False)
    U = Q @ U_tilde
    return U, s, Vh

In [4]:
def trials():
    num_trials = 10
    ks = 10
    
    errors = np.zeros([ks, num_trials])
    
    for k in tqdm(range(1,ks+1)):
        for trial in range(num_trials):
    
            m = 10
            n = 10000
            M = np.random.choice([0,1], size=[m, n], p=[0.8,0.2])
            U, s, Vh = randsvd(M, k=k)
            error = lg.norm(M - U @ np.diag(s) @ Vh) / M.size
            errors[k-1, trial] = error
    
    return M, np.real(U @ np.diag(s) @ Vh), errors

In [ ]:
B, B_rec, errors_ar = trials()

  0%|          | 0/10 [00:00<?, ?it/s]

Getting OPU data



OPU transform:   0%|          | 0/231 [00:00<?, ?it/s]

OPU: random projections of an array of size (231,10000)



OPU transform: 100%|██████████| 231/231 [00:00<00:00, 235.30it/s]

...

(231, 6)
255
5
Got OPU data
Getting OPU data



OPU transform:   0%|          | 0/231 [00:00<?, ?it/s]

OPU: random projections of an array of size (231,10000)



OPU transform: 100%|██████████| 231/231 [00:00<00:00, 233.90it/s]

(231, 7)
255
5
Got OPU data
Getting OPU data


In [ ]:
errors = np.mean(errors_ar, axis=1)
plt.rcParams.update({'font.size': 14})
plt.figure(figsize=(3.5,3.5))
plt.plot(np.linspace(1,10, num=10), errors)
plt.yscale('log')
plt.ylabel('Average error per entry')
plt.xlabel('Number of projections')
plt.tight_layout()
plt.savefig('svd_OPU_recon_error.pdf')
print (errors)

In [ ]:
from lightonml.datasets import MNIST
from lightonml.encoding.base import BinaryThresholdEncoder

def mnist_experiment():
    # load data
    (_, _), (X, _) = MNIST()
    # encode data
    encoder = BinaryThresholdEncoder()
    X_encoded = encoder.transform(X)
    
    return X_encoded[:500]

In [ ]:
M = mnist_experiment()
U, s, Vh = randsvd(M, k=np.min(M.shape))
error = lg.norm(M - U @ np.diag(s) @ Vh) / M.size

In [ ]:
U_true, s_true, Vh_true = np.linalg.svd(M, full_matrices=False)

In [ ]:
n_svecs = 7

plt.rcParams.update({'font.size': 9})
plt.figure(figsize=(8, 2.5));
for i in range(n_svecs):
    fig = plt.subplot(2,n_svecs, 1+i)
    phase = (Vh[i, 100]/ Vh_true[i, 100]) # rotation of system may be required
    img = np.real(np.conj(phase)*Vh[i]).reshape([28,28])
    plt.imshow(img, cmap='gray')
    plt.xticks([])
    plt.yticks([])
    
    fig = plt.subplot(2,n_svecs, n_svecs + 1+i)
    img_true = (Vh_true[i]).reshape([28,28])
    plt.imshow(img_true, cmap='gray')
    plt.xticks([])
    plt.yticks([])
    rel_error = 100*np.linalg.norm(img-img_true) / np.linalg.norm(img_true)
    plt.title('{:.2e}'.format(rel_error), y=-0.4)

plt.tight_layout()